In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report

2024-08-29 15:33:20.859226: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-29 15:33:20.866653: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-29 15:33:20.874805: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-29 15:33:20.877247: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-29 15:33:20.883856: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Carregando os dados
resultados = pd.read_csv('resultados_modelo.csv')

/tmp/ipykernel_125398/727491634.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  resultados = pd.read_csv('resultados_modelo.csv')


In [3]:
falhas = pd.read_csv('falhas_modelo.csv')

/tmp/ipykernel_125398/3729996659.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  falhas = pd.read_csv('falhas_modelo.csv')


In [4]:
resultados.head()

,KNR,NAME,ID,STATUS,UNIT,VALUE_ID,VALUE,DATA,day,month,year,hour,minute,second
0,2023-5076008,255,718,13,7,80,0.007334,2024-02-02 08:03:03,2.0,2.0,2024.0,8.0,3.0,3.0
1,2023-5076008,255,718,13,7,80,0.007341,2024-02-02 07:37:59,2.0,2.0,2024.0,7.0,37.0,59.0
2,2023-5076008,255,718,10,7,80,0.007357,2024-02-02 08:32:14,2.0,2.0,2024.0,8.0,32.0,14.0
3,2023-5076015,255,718,13,7,80,0.007322,2024-02-07 14:05:16,7.0,2.0,2024.0,14.0,5.0,16.0
4,2023-5076015,255,718,10,7,80,0.007352,2024-02-07 14:31:41,7.0,2.0,2024.0,14.0,31.0,41.0


In [5]:
# Convertendo as colunas de data e hora para um único datetime
resultados['DATA'] = pd.to_datetime(resultados[['day', 'month', 'year', 'hour', 'minute', 'second']])

In [6]:
resultados = resultados.drop(['day', 'month', 'year', 'hour', 'minute', 'second'], axis=1)

In [7]:
falhas.head()

,KNR,MODELO,COR,MOTOR,ESTACAO,USUARIO,HALLE,FALHA,S_GROUP_ID,day,month,year,hour,minute,second
0,2023-2056234,T-CROSS,2R2R,DHS,HCEQS506,4013334,ZP7,paralama dianteiro 01. amassado,5,20,5,2024,21,31,51
1,2023-2056234,T-CROSS,2R2R,DHS,HCEQS506,4013334,ZP7,porta traseira 04. caroço / pico,4,20,5,2024,21,30,22
2,2023-2056234,T-CROSS,2R2R,DHS,PCEQS100,4004554,ZP5,porta dianteira solda - respingo,4,6,12,2023,6,48,29
3,2023-2056234,T-CROSS,2R2R,DHS,PCEQS114,2020297,ZP5,cinto de segurança - fixação assoalho rosca nok,2,5,12,2023,21,36,33
4,2023-2056234,T-CROSS,2R2R,DHS,PCEQS620,2058570,ZP6,módulo do acc (13) peça incorreta ¨,9830946,20,5,2024,20,4,55


In [8]:
falhas['DATA'] = pd.to_datetime(falhas[['day', 'month', 'year', 'hour', 'minute', 'second']])

In [9]:
falhas = falhas.drop(['day', 'month', 'year', 'hour', 'minute', 'second', 'MODELO', 'COR', 'MOTOR', 'ESTACAO', 'USUARIO', 'HALLE', 'FALHA'], axis=1)

In [10]:
resultados['KNR'] = resultados['KNR'].astype(str)
falhas['KNR'] = falhas['KNR'].astype(str)

In [11]:
merged_data = pd.merge(resultados, falhas, on=['KNR'], how='outer')
merged_data

,KNR,NAME,ID,STATUS,UNIT,VALUE_ID,VALUE,DATA_x,S_GROUP_ID,DATA_y
0,2023-2016173,59.0,2.0,10.0,4.0,1,0.006793,2023-06-06 07:07:30,NaN,NaT
1,2023-2016173,59.0,2.0,10.0,4.0,3,0.018682,2023-06-06 07:07:30,NaN,NaT
2,2023-2016173,59.0,2.0,10.0,5.0,0,0.006842,2023-06-06 07:07:30,NaN,NaT
3,2023-2016173,59.0,2.0,10.0,5.0,2,0.007029,2023-06-06 07:07:30,NaN,NaT
4,2023-2016173,219.0,2.0,10.0,4.0,3,0.007359,2023-06-06 07:07:10,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...
27303651,2024-3716002,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2,2024-08-15 15:12:48
27303652,2024-3716002,NaN,NaN,NaN,NaN,NaN,NaN,NaT,4,2024-08-15 15:11:53
27303653,2024-3716002,NaN,NaN,NaN,NaN,NaN,NaN,NaT,5,2024-07-31 15:40:07
27303654,2024-3876001,NaN,NaN,NaN,NaN,NaN,NaN,NaT,140,2024-07-11 13:34:34


In [12]:
# Verificando a fusão
merged_data.head()

,KNR,NAME,ID,STATUS,UNIT,VALUE_ID,VALUE,DATA_x,S_GROUP_ID,DATA_y
0,2023-2016173,59.0,2.0,10.0,4.0,1,0.006793,2023-06-06 07:07:30,NaN,NaT
1,2023-2016173,59.0,2.0,10.0,4.0,3,0.018682,2023-06-06 07:07:30,NaN,NaT
2,2023-2016173,59.0,2.0,10.0,5.0,0,0.006842,2023-06-06 07:07:30,NaN,NaT
3,2023-2016173,59.0,2.0,10.0,5.0,2,0.007029,2023-06-06 07:07:30,NaN,NaT
4,2023-2016173,219.0,2.0,10.0,4.0,3,0.007359,2023-06-06 07:07:10,NaN,NaT


In [13]:
merged_data.dropna()

,KNR,NAME,ID,STATUS,UNIT,VALUE_ID,VALUE,DATA_x,S_GROUP_ID,DATA_y
2115,2023-2056234,0.0,1.0,10.0,3.0,5,0.742781,2024-05-16 13:21:03,5,2024-05-20 21:31:51
2116,2023-2056234,0.0,1.0,10.0,3.0,5,0.742781,2024-05-16 13:21:03,4,2024-05-20 21:30:22
2117,2023-2056234,0.0,1.0,10.0,3.0,5,0.742781,2024-05-16 13:21:03,4,2023-12-06 06:48:29
2118,2023-2056234,0.0,1.0,10.0,3.0,5,0.742781,2024-05-16 13:21:03,2,2023-12-05 21:36:33
2119,2023-2056234,0.0,1.0,10.0,3.0,5,0.742781,2024-05-16 13:21:03,9830946,2024-05-20 20:04:55
...,...,...,...,...,...,...,...,...,...,...
26899302,2024-1446373,40.0,2.0,10.0,4.0,3,0.008477,2024-04-08 10:47:52,5,2024-04-08 16:03:06
26899303,2024-1446373,40.0,2.0,10.0,4.0,3,0.008477,2024-04-08 10:47:52,1,2024-04-08 16:06:43
26899304,2024-1446373,40.0,2.0,10.0,4.0,3,0.008477,2024-04-08 10:47:52,2,2024-04-08 16:13:50
26899305,2024-1446373,40.0,2.0,10.0,4.0,3,0.008477,2024-04-08 10:47:52,5,2024-04-04 19:57:09


In [14]:
merged_data = merged_data.drop(['DATA_x', 'DATA_y'], axis=1)

In [15]:
merged_data.dropna()

,KNR,NAME,ID,STATUS,UNIT,VALUE_ID,VALUE,S_GROUP_ID
2115,2023-2056234,0.0,1.0,10.0,3.0,5,0.742781,5
2116,2023-2056234,0.0,1.0,10.0,3.0,5,0.742781,4
2117,2023-2056234,0.0,1.0,10.0,3.0,5,0.742781,4
2118,2023-2056234,0.0,1.0,10.0,3.0,5,0.742781,2
2119,2023-2056234,0.0,1.0,10.0,3.0,5,0.742781,9830946
...,...,...,...,...,...,...,...,...
26899302,2024-1446373,40.0,2.0,10.0,4.0,3,0.008477,5
26899303,2024-1446373,40.0,2.0,10.0,4.0,3,0.008477,1
26899304,2024-1446373,40.0,2.0,10.0,4.0,3,0.008477,2
26899305,2024-1446373,40.0,2.0,10.0,4.0,3,0.008477,5


In [33]:
# Assuming 'merged_data' is your DataFrame
features = merged_data[['KNR', 'NAME', 'ID', 'STATUS', 'UNIT', 'VALUE_ID']]
target = merged_data['S_GROUP_ID']


In [34]:
features

,KNR,NAME,ID,STATUS,UNIT,VALUE_ID
0,2023-2016173,59.0,2.0,10.0,4.0,1
1,2023-2016173,59.0,2.0,10.0,4.0,3
2,2023-2016173,59.0,2.0,10.0,5.0,0
3,2023-2016173,59.0,2.0,10.0,5.0,2
4,2023-2016173,219.0,2.0,10.0,4.0,3
...,...,...,...,...,...,...
27303651,2024-3716002,NaN,NaN,NaN,NaN,NaN
27303652,2024-3716002,NaN,NaN,NaN,NaN,NaN
27303653,2024-3716002,NaN,NaN,NaN,NaN,NaN
27303654,2024-3876001,NaN,NaN,NaN,NaN,NaN


In [35]:
# Option 1: Drop rows with NaN values
features = features.dropna()
target = target[features.index]

In [36]:
print(f"Features shape: {features.shape}")
print(f"Target shape: {target.shape}")



Features shape: (26621798, 6)
Target shape: (26621798,)


In [39]:
features = pd.get_dummies(features, drop_first=True)


MemoryError: Unable to allocate 223. GiB for an array with shape (26621798, 9009) and data type bool

In [37]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [38]:
scaler = StandardScaler()

# Fit only on training data to prevent data leakage
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


ValueError: could not convert string to float: '2023-4916275'

In [27]:
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Binary classification

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


/home/fillmore/venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1724955490.499597  124930 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-29 15:18:10.501815: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Sk

In [28]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)


ValueError: could not convert string to float: '#MULTIVALUE'